# Final Assessment Scratch Pad

## Instructions

1. Please use only this Jupyter notebook to work on your model, and **do not use any extra files**. If you need to define helper classes or functions, feel free to do so in this notebook.
2. This template is intended to be general, but it may not cover every use case. The sections are given so that it will be easier for us to grade your submission. If your specific use case isn't addressed, **you may add new Markdown or code blocks to this notebook**. However, please **don't delete any existing blocks**.
3. If you don't think a particular section of this template is necessary for your work, **you may skip it**. Be sure to explain clearly why you decided to do so.

## Report

##### Overview
**[TODO]**
From my analysis of the data set given to us, we are expected to carry out a classification of the images according the labels. My approach to this will be to construct a CNN model as of what I have learnt from the NN lectures.


##### 1. Descriptive Analysis
**[TODO]**
The first thing I did is to analyse the data given, both images and labels. Firstly, I identified the data type of values in both images and labels are both float16. Next, I identified that range of values of each image and the range of labels. With that I went on to find out the mean and standard deviation of the values of each image. This, with the range, I am able to have some hints if there are outliers in the data. For example, both mean contains nan, meaning there must be nan in both labels and images.

##### 2. Detection and Handling of Missing Values
**[TODO]**
By using numpy, I am able to find out the number of unique labels and the count of each unique lable. The same for images, I used numpy to find out the images with nan values. To handle image nan values, I used a method 'image_replacing_nan' which replaces the nans in each image with the mean value (for separate channels) of each image ignoring nans.  For the nans in labels, I wrote a method 'image_label_filter_nan' which drops the nan of labels and the corrospoing image at that particular indexes.

##### 3. Detection and Handling of Outliers
**[TODO]**
I clipped the values of all images to be within [0, 255] such that it adhere to the standard 8-bit per channel format, where pixel values must be within this range. I also normalised the pixel values of all images by images/255, scaling the values to be within 0 and 1, which improves the numberical stabilities in further operations and training. 

##### 4. Detection and Handling of Class Imbalance 
**[TODO]**
Identified that label 2 is severely imbalanced. After some research, instead of oversampling, which uses too much time. I calculated class weights, which  can be used in torch.nn.CrossEntropyLoss by passing them as an argument. This helps to counteract the effect of class imbalance by making the loss function penalize misclassifications of the minority class more heavily than those of the majority class.

##### 5. Understanding Relationship Between Variables

**[TODO]**

##### 6. Data Visualization

**[TODO]** 


##### 7. General Preprocessing
**[TODO]**
This is done by  handling missing values, clipping and normalizing images, and filtering out NaN values in labels.  

##### 8. Feature Selection 
I tried to select the most important features of the the images through PCA as part of preprocessing. However, the F1 scored received after was worse than one with out PCA. Hence PCA is not adopted. 
**[TODO]**

##### 9. Feature Engineering
**[TODO]**
image augmentation techniques like random flips, rotations, and affine transformations. This not only enriches the dataset but also makes the model more robust to variations in new data.

##### 10. Creating Models
**[TODO]**
The CNN architecture includes convolutional layers, batch normalization, Leaky ReLU activation, max pooling, dropout for regularization, and fully connected layers for classification. 
##### 11. Model Evaluation
**[TODO]**
The model is evaluated using the F1 scores, which is better for classification that involves class imbalance. F1 ensures that both the precision (the accuracy of positive predictions) and recall (the ability to detect positive instances) are considered. 
I further went on to split the data into training set, validation set and test set. This is such that the best hyperparameters from prediciting using validation set is are used on the test set.
##### 12. Hyperparameters Search
**[TODO]**
The hyperparameter search is done on the learning rate and the drop out probability by looping through a few possible choices and choose the ones with the best F1. 
##### Conclusion
**[TODO]**
In conclusion, the CNN used to classify the images according to the labels 
is done with thorough data processing, training with different sets of data to have finally reach a reasonably sound F1 score, indicating that the model is correctly classifying the images majority of the times. 




GPT links
https://chat.openai.com/share/a1203a1f-b270-4230-a750-9205d12b405e
https://chat.openai.com/share/87142aab-99cc-4ad8-ac6a-df477c4bd68a


---

# Workings (Not Graded)

You will do your working below. Note that anything below this section will not be graded, but we might counter-check what you wrote in the report above with your workings to make sure that you actually did what you claimed to have done. 

## Import Packages

Here, we import some packages necessary to run this notebook. In addition, you may import other packages as well. Do note that when submitting your model, you may only use packages that are available in Coursemology (see `main.ipynb`).

In [2]:
import pandas as pd
import os
import numpy as np
from matplotlib import pyplot as plt
import torch
import torch.nn as nn 
import torch.nn.functional as F

## Load Dataset

The dataset `data/images.npy` is of size $(N, C, H, W)$, where $N$, $C$, $H$, and $W$ correspond to the number of data, image channels, image width, and image height, respectively.

A code snippet that loads the data is provided below.

### Load Image Data

In [3]:
with open('data.npy', 'rb') as f:
    data = np.load(f, allow_pickle=True).item()
    images = data['image']
    labels = data['label']

print(images.dtype)
print(labels.dtype)
print('Shape:', images.shape)

float16
float16
Shape: (2911, 3, 16, 16)


## Data Exploration & Preparation

### 1. Descriptive Analysis

In [14]:
image_mean = np.mean(images, axis=(0, 2,3))
print("image mean: ", image_mean)
label_mean = np.mean(labels, axis=0)
print("label mean: ", label_mean)
print("image std:", np.std(images, axis=(0, 2, 3)))


image mean:  [nan nan nan]
label mean:  nan
image std: [nan nan nan]


### 2. Detection and Handling of Missing Values

In [4]:
unique_labels, counts = np.unique(labels, return_counts=True)

# Now, unique_labels holds the unique elements, and counts holds the corresponding count for each unique element
for label, count in zip(unique_labels, counts):
    print(f"Label: {label}, Count: {count}")
nan_counts_per_image = [np.sum(np.isnan(image)) for image in images]

# Optionally, print the count for each image
for index, count in enumerate(nan_counts_per_image):
    print(f"Image {index} has {count} NaN values")
'''
for index, image in enumerate(images):
    print(f"Image {index} has data type: {image.dtype}")
'''


Label: 0.0, Count: 2392
Label: 1.0, Count: 203
Label: 2.0, Count: 25
Label: nan, Count: 291
Image 0 has 11 NaN values
Image 1 has 7 NaN values
Image 2 has 2 NaN values
Image 3 has 13 NaN values
Image 4 has 5 NaN values
Image 5 has 6 NaN values
Image 6 has 6 NaN values
Image 7 has 6 NaN values
Image 8 has 4 NaN values
Image 9 has 5 NaN values
Image 10 has 14 NaN values
Image 11 has 8 NaN values
Image 12 has 4 NaN values
Image 13 has 6 NaN values
Image 14 has 8 NaN values
Image 15 has 6 NaN values
Image 16 has 9 NaN values
Image 17 has 11 NaN values
Image 18 has 7 NaN values
Image 19 has 6 NaN values
Image 20 has 5 NaN values
Image 21 has 4 NaN values
Image 22 has 13 NaN values
Image 23 has 9 NaN values
Image 24 has 4 NaN values
Image 25 has 8 NaN values
Image 26 has 7 NaN values
Image 27 has 8 NaN values
Image 28 has 7 NaN values
Image 29 has 5 NaN values
Image 30 has 8 NaN values
Image 31 has 7 NaN values
Image 32 has 7 NaN values
Image 33 has 12 NaN values
Image 34 has 11 NaN values
I

'\nfor index, image in enumerate(images):\n    print(f"Image {index} has data type: {image.dtype}")\n'

### 3. Detection and Handling of Outliers

### 4. Detection and Handling of Class Imbalance

### 5. Understanding Relationship Between Variables

### 6. Data Visualization

## Data Preprocessing

### 7. General Preprocessing

### 8. Feature Selection

### 9. Feature Engineering

## Modeling & Evaluation

### 10. Creating models

### 11. Model Evaluation

In [ ]:
def split_data(images, labels, test_size=0.2, val_size=0.25):

    # Split data into training and test set
    X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=test_size, random_state=42, stratify=labels)

    # Split training data into training and validation set
    val_size_adjusted = val_size / (1 - test_size)  # Adjust validation size based on remaining dataset
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_size_adjusted, random_state=42, stratify=y_train)

    return X_train, X_val, X_test, y_train, y_val, y_test


nan_indices = np.argwhere(np.isnan(y)).squeeze()
mask = np.ones(y.shape, bool)
mask[nan_indices] = False
filtered_images = X[mask]
filtered_labels = y[mask]

### 12. Hyperparameters Search

In [ ]:
best_f1 = 0.0
best_param = {}
for lr in [1e-4, 1e-3, 5e-3]:
    for drop_prob in [0.3, 0.4, 0.5]:
        model = Model(lr=lr, drop_prob=drop_prob)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        f1 = f1_score(y_val, y_pred, average='macro')
        if f1 > best_f1:
            best_f1 = f1
            best_param = {'lr': lr, 'drop_prob': drop_prob}
print("Best Parameters:", best_param)